# Cell type identification from scRNA-seq data using ACTINN

## Why is RNA important?

To understand what scRNA-seq data represents, let's first explore the role of RNA (Ribonucleic Acid) inside cells. All cells in our body, regardless of whether they make up tissues in the heart, brain, or skin, contain the same genetic code (DNA). Yet, despite having identical DNA, cells look different, perform distinct functions, and form specialized tissues and organs.

How is this possible?

The key lies in differential [gene expression](https://en.wikipedia.org/wiki/Gene_expression), the process by which genetic instructions are used to synthesize gene products like proteins. DNA acts as the blueprint, but it is RNA, specifically messenger RNA (mRNA), that translates these instructions into action. Different types of cells transcribe distinct subsets of genes into mRNAs, leading to the synthesis of unique sets of proteins. These distinct protein profiles allow cells to specialize and perform various functions within the body [1,2].

Messenger RNA (mRNA) acts as a critical intermediary, carrying genetic instructions from DNA to ribosomes,the cellular machinery responsible for protein synthesis.

Proteins like enzymes drive metabolic pathways, while structural proteins such as actin and myosin give muscles their ability to contract and move. Cytoskeletal proteins help maintain cell shape and integrity.

Therefore, the unique set of mRNA molecules present in a cell at any given moment(its **transcriptome**) reveals which genes are "turned on." This transcriptome dictates which proteins the cell is making, which in turn defines the cell's type and function. Measuring this transcriptome is the fundamental goal of single-cell RNA sequencing (scRNA-seq).


## Single-cell RNA sequencing vs Bulk RNA sequencing

RNA-Seq (short for RNA sequencing) is a technique used to quantify and identify RNA molecules in a biological sample, providing a snapshot of the transcriptome at a specific time. Sequencing of RNA can be mainly conducted in two ways: Either by sequencing the mixed RNA from the source of interest across cells (bulk sequencing) or by sequencing the transcriptomes of the cells individually (single-cell sequencing).

Bulk RNA-Seq results in cell-averaged expression profiles, which are generally easier to analyze, but also hide some of the complexity such as cell expression profile heterogeneity, which may help answer the question of interest. Some drugs or perturbations may affect only specific cell types or interactions between cell types.

For example, in oncology, it is possible to have rare drug resistant tumor cells causing relapse, which is difficult to identify by simple bulk RNA-seq even on cultured cells.To uncover such relationships, it is vital to examine gene expression on a single-cell level.[3]

## Libraries commonly used for scRNA-seq analysis

[ScanPy](https://scanpy.readthedocs.io/en/stable/) is a scalable toolkit for analyzing single-cell gene expression data. It includes methods for preprocessing, visualization, clustering, pseudotime and trajectory inference, differential expression testing, and simulation of gene regulatory networks.

[ANNDATA](https://github.com/theislab/anndata) was presented alongside ScanPy as a generic class for handling annotated data matrices that can deal with the sparsity inherent in gene expression data.

This is also a good point to explore DeepChem's [tutorial](https://deepchem.io/tutorials/scanpy/) on building an scRNA-seq analysis pipeline using ScanPy.




## ACTINN (automated identification of cell types in single cell RNA sequencing )

This tutorial guides you through working with scRNA-seq data and demonstrates how to train and evaluate a model using DeepChem. The objective is to replicate the experiments performed for cell type identification in [ACTINN](https://academic.oup.com/bioinformatics/article/36/2/533/5540320).

Cell type identification is one of the major goals in single cell RNA sequencing (scRNA-seq). Traditional methods for assigning cell types typically involve the use of unsupervised clustering, the identification of signature genes in each cluster, followed by a manual lookup of these genes in the literature and databases to assign cell types. However, there are several limitations associated with these approaches, such as unwanted sources of variation that influence clustering and a lack of canonical markers for certain cell types.[1]

ACTINN employs a neural network with three hidden layers, trains on datasets with predefined cell types and predicts cell types for other datasets based on the trained parameters.

Here, ACTINN is implemented with a deepchem wrapper.

## Colab

This tutorial and the rest in this sequence can be done in Google colab. If you'd like to open this notebook in colab, you can use the following link.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepchem/deepchem/blob/master/examples/tutorials/Cell_type_identification_using_scRNAseq_data.ipynb)

## Setup

To run DeepChem within Colab, you'll need to run the following installation commands. You can of course run this tutorial locally if you prefer. In that case, don't run these cells since they will download and install DeepChem in your local machine again.

In [1]:
#!pip install deepchem
import deepchem as dc
dc.__version__

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
No normalization for NumAmideBonds. Feature removed!
No normalization for NumAtomStereoCenters. Feature removed!
No normalization for NumBridgeheadAtoms. Feature removed!
No normalization for NumHeterocycles. Feature removed!
No normalization for NumSpiroAtoms. Feature removed!
No normalization for NumUnspecifiedAtomStereoCenters. Feature removed!
No normalization for Phi. Feature removed!


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/Users/harin/Desktop/deepchem/deepchem/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


'2.8.1.dev'

## Data Collection and Splitting




A subset of the 'tma_ss2_cleaned' dataset from this [source](https://figshare.com/articles/ACTINN/8967116), which was originally used in ACTINN, is utilized here.

There are two approaches to creating the train and test sets:
1. Splitting a single dataset into training and testing portions.
2. Using datasets from different sources as the train and test sets.


In [4]:
import os
import pandas as pd
import numpy as np
current_dir = os.path.dirname(os.path.realpath('__file__'))

In [5]:
file_names = ['train_sample.h5', 'train_label_sample.txt', 'test_sample.h5', 'test_label_sample.txt']
base_url = 'https://raw.githubusercontent.com/Harindhar10/deepchem/actinn_tutorial/examples/tutorials/assets/scRNAseq'

for i in range(len(file_names)):
  dc.utils.download_url(
      f"{base_url}/{file_names[i]}",
      current_dir,
      f"{file_names[i]}"
  )

### 1. Same source

In [6]:
labels = pd.read_csv(os.path.join(current_dir,'train_label_sample.txt'), header=None, sep='\t') 
dataset = pd.read_hdf(os.path.join(current_dir,'train_sample.h5'))

In [7]:
dataset

,tma_facs_14301,tma_facs_4756,tma_facs_14382,tma_facs_423,tma_facs_15612,tma_facs_4729,tma_facs_8174,tma_facs_9855,tma_facs_9055,tma_facs_8668,...,tma_facs_5182,tma_facs_19106,tma_facs_11386,tma_facs_14006,tma_facs_7477,tma_facs_15352,tma_facs_1314,tma_facs_2172,tma_facs_1625,tma_facs_12420
Cts7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sdr42e1,0,0,0,0,0,0,0,46,52,0,...,0,0,0,0,0,0,0,0,0,0
Commd5,0,0,0,0,0,0,0,89,0,0,...,0,0,0,0,0,0,0,0,50,0
Fam170a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Olfr748,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Bcar3,0,0,0,0,0,0,124,34,0,0,...,0,0,0,0,0,0,8,0,0,0
Hk1,0,0,5,0,2,0,71,149,37,0,...,0,0,10,0,0,149,0,0,3,0
Sh3glb1,18,0,10,0,6,678,205,79,86,4,...,187,0,408,69,12,0,11,0,13,36
Calr4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
labels # Columns: cell ID and cell type

,0,1
0,tma_facs_14301,Monocyte
1,tma_facs_4756,Endothelial cell
2,tma_facs_14382,Monocyte
3,tma_facs_423,B cell
4,tma_facs_15612,Monocyte
...,...,...
3995,tma_facs_15352,Monocyte
3996,tma_facs_1314,B cell
3997,tma_facs_2172,B cell
3998,tma_facs_1625,B cell


In [9]:
n_types = 0 # To store the number of cell types present in the dataset, which is need to define the output layer of the model later on.

# Convertes cell type labels from string ('B Cell', 'T Cell' etc) to integers (1, 2 etc)
def convert_type2label(types):
    global n_types
    types = list(types)
    unique_types = sorted(set(types)) # Sorting ensures consistent label order
    n_types = len(unique_types)

    type_to_label_dict = {t: i for i, t in enumerate(unique_types)}
    labels = np.array([type_to_label_dict[t] for t in types])

    return labels

labels_int = convert_type2label(labels[1])

In [10]:
# Reshaping labels from (n,) to (n, 1) to be compatible with DeepChem's Dataset class
labels_int = labels_int.reshape(labels_int.shape[0],1)

In [11]:
# gene IDs are stored
train_genes = dataset.index.to_numpy()

# genes x cells (features x samples) -> cells x genes (samples x features)
dataset = np.transpose(dataset)

# Creating Deepchem 'Dataset' object
dataset = dc.data.NumpyDataset(X=np.array(dataset,dtype=np.float32),y = labels_int, ids= labels[0])

In [12]:
splitter = dc.splits.SingletaskStratifiedSplitter()
train_dataset, test_dataset = splitter.train_test_split(dataset)

### 2. Different sources

When using datasets from different sources, the gene sets may not completely overlap. Since the ACTINN model defines its layers based on the gene set, it's essential that both the training and testing sets contain the same genes. To ensure this, we first identified the genes common to both datasets and filtered out those that were not shared.

We will now demonstrate how to prepare data from different sources, which is the method we'll use for the rest of this tutorial

In [164]:
train_set = pd.read_hdf(os.path.join(current_dir,'train_sample.h5'))
train_labels = pd.read_csv(os.path.join(current_dir,'train_label_sample.txt'), header=None, sep='\t')
test_set = pd.read_hdf(os.path.join(current_dir,'test_sample.h5'))
test_labels = pd.read_csv(os.path.join(current_dir,'test_label_sample.txt'), header=None, sep='\t')

common_genes = train_set.index.intersection(test_set.index)
common_genes = sorted(common_genes)

train_set = train_set.loc[common_genes]
test_set = test_set.loc[common_genes]

Convertes cell type labels from string ('B Cell', 'T Cell' etc) to integers (1, 2 etc)

In [ ]:
train_labels_int = convert_type2label(train_labels[1])
test_labels_int = convert_type2label(test_labels[1])

# Reshaping labels from (n,) to (n, 1) to be compatible with DeepChem's Dataset class
train_labels_int = train_labels_int.reshape(train_labels_int.shape[0],1)
test_labels_int = test_labels_int.reshape(test_labels_int.shape[0],1)

In [166]:
# gene IDs are stored
train_genes = train_set.index.to_numpy()
test_genes = test_set.index.to_numpy()

In [167]:
print(len(train_genes),len(test_genes))

2000 2000


In [168]:
# genes x cells (features x samples) -> cells x genes (samples x features)
train_set = np.transpose(train_set)
test_set = np.transpose(test_set)

In [169]:
# Creating Deepchem 'Dataset' object
train_dataset = dc.data.NumpyDataset(X=np.array(train_set,dtype=np.float32),y = train_labels_int, ids= train_labels[0])
test_dataset = dc.data.NumpyDataset(X=np.array(test_set,dtype=np.float32), y = test_labels_int, ids= test_labels[0])

In [170]:
train_dataset

<NumpyDataset X.shape: (4000, 2000), y.shape: (4000, 1), w.shape: (4000, 1), task_names: [0]>

In [171]:
test_dataset

<NumpyDataset X.shape: (1000, 2000), y.shape: (1000, 1), w.shape: (1000, 1), task_names: [0]>

## Train set data transformation and gene filtering

Next, each cell’s expression value was normalized to its total expression value and multiplied by a scale factor of 10 000.

In [172]:
# 1) extract gene names & data array
gene_names = train_genes
X = train_dataset.X

# 2) library-size normalize to 10,000 (in-place)
row_sums = X.sum(axis=1, keepdims=True)  # shape (n_cells, 1)
X /= row_sums # broadcast divide
X *= 10000

In [173]:
X.shape # (number of cells, number of genes)

(4000, 2000)

The counts were increased by 1, and the log2 value was calculated

In [174]:
# 3) log2(x + 1) transform
X = np.log2(X + 1)

To filter out outlier genes, the genes with the highest 1% and lowest 1% expression were removed.


In [175]:
expr = X.sum(axis=0)
low, high = np.percentile(expr, [1, 99])
mask_expr = (expr >= low) & (expr <= high)

X = X[:, mask_expr]
gene_names = gene_names[mask_expr]


To remove genes with zero mean (genes that aren't expressed in any of the cells)

In [176]:
mean_expr = X.mean(axis=0)
mask_mean = mean_expr > 0
X = X[:, mask_mean]
gene_names = gene_names[mask_mean]

The gene with the highest 1% and the lowest 1% standard deviation are also removed.

In [177]:
mean_expr = X.mean(axis=0)
cv = X.std(axis=0) / mean_expr
low_cv, high_cv = np.percentile(cv, [1, 99])
mask_cv = (cv >= low_cv) & (cv <= high_cv)

X = X[:, mask_cv]
gene_names = gene_names[mask_cv]
train_genes = gene_names

In [178]:
# Creating Deepchem 'Dataset' object
train_dataset = dc.data.NumpyDataset(X=X,y = train_labels_int, ids= train_labels[0])

In [179]:
train_dataset

<NumpyDataset X.shape: (4000, 1865), y.shape: (4000, 1), w.shape: (4000, 1), task_names: [0]>

## Test set transformation and filtering

The exact same normalisation aproach used for train set is used for test set.
First, each cell’s expression value was normalized to its total expression value and multiplied by a scale factor
of 10 000. The counts were increased by 1, and the log2 value was
calculated

In [180]:
# 1) extract data array
X = test_dataset.X

# 2) library-size normalize to 10,000 (in-place)
row_sums = X.sum(axis=1, keepdims=True)  # shape (1, n_cells)
X /= row_sums  # broadcast divide
X *= 10000

# 3) log2(x + 1) transform (in-place)
X = np.log2(X + 1, out=X)


In [181]:
len(train_genes)

1865

In [182]:
len(test_genes)

2000


The gene list obtained from the train set filtering steps is used to mask the test set. In the original ACTINN implementation, genes were filtered using both the training and test sets. To avoid potential information leakage, we instead apply filtering based solely on the training set.

In [183]:
test_gene_mask=[i in train_genes for i in test_genes]
X = X[:,test_gene_mask]

In [184]:
test_dataset = dc.data.NumpyDataset(X=np.array(X,dtype=np.float32), y=test_labels_int, ids=test_labels[0])

In [185]:
test_dataset

<NumpyDataset X.shape: (1000, 1865), y.shape: (1000, 1), w.shape: (1000, 1), task_names: [0]>

ACTINN implemented with deepchem Model class as wrapper

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from deepchem.models.torch_models import TorchModel
from deepchem.models.losses import SparseSoftmaxCrossEntropy
from deepchem.models.optimizers import Adam
from deepchem.models.optimizers import ExponentialDecay
from deepchem.metrics import from_one_hot


class ActinnClassifier(nn.Module):

    def __init__(self, output_dim=None, input_size=None):

        if output_dim == None or input_size == None:
            raise ValueError('Must explicitly declare input dim (num features) and output dim (number of classes)')

        super(ActinnClassifier, self).__init__()
        self.inp_dim = input_size
        self.out_dim = output_dim

        # feed forward layers
        self.classifier = nn.Sequential(
                                        nn.Linear(self.inp_dim, 100),
                                        nn.ReLU(),

                                        nn.Linear(100, 50),
                                        nn.ReLU(),

                                        nn.Linear(50, 25),
                                        nn.ReLU(),

                                        nn.Linear(25, output_dim)
                                        )

    def forward(self, x):
        """
        Forward pass of the classifier
        """
        logits = self.classifier(x)
        probabilities = F.softmax(logits, dim=1)
        predictions = from_one_hot(probabilities.cpu().detach(), axis=1)
        return (predictions, logits)
    

class ACTINNModel(TorchModel):
    def __init__(self, output_dim = None, input_size = None, **kwargs):

        self.model = ActinnClassifier(output_dim, input_size)
        self.criterion = SparseSoftmaxCrossEntropy()
        print('model', self.model)
        cf_optimizer = Adam(learning_rate=0.0001,
                            beta1=0.9,
                            beta2=0.999,
                            epsilon=1e-08,
                            weight_decay=0.005,
                            )

        cf_decayRate = 0.95
        cf_lr_scheduler = ExponentialDecay(initial_rate=0.0001, decay_rate=cf_decayRate, decay_steps=1000)
        super(ACTINNModel,
              self).__init__(self.model,
                             loss=self.criterion,
                             optimizer=cf_optimizer,
                             learning_rate=cf_lr_scheduler,
                             output_types=['prediction', 'loss'],
                             **kwargs)


## Model training and evaluation

In [160]:
model = ACTINNModel(output_dim= n_types,input_size= len(train_genes))

model ActinnClassifier(
  (classifier_sequential): Sequential(
    (0): Linear(in_features=1865, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=25, bias=True)
    (5): ReLU()
    (6): Linear(in_features=25, out_features=12, bias=True)
  )
)


In [161]:
model.fit(train_dataset, nb_epoch=7)
predictions = model.predict(test_dataset)

In [162]:
classification_metric = dc.metrics.Metric(dc.metrics.accuracy_score)
scores = model.evaluate(test_dataset, [classification_metric], n_classes=n_types)

In [163]:
print(scores['accuracy_score'])

0.971


### References
[1] Tang, F., Barbacioru, C., Wang, Y., Nordman, E., Lee, C., Xu, N., ... & Surani, M. A. (2009). mRNA-Seq whole-transcriptome analysis of a single cell. Nature methods, 6(5), 377-382.

[2] Alberts, B., Johnson, A., Lewis, J., Morgan, D., Raff, M., Roberts, K., & Walter, P. (2014). Molecular Biology of the Cell (6th ed.). Garland Science.

[3] Jorge A. Tzec-Interián, Daianna González-Padilla, Elsa B. Góngora-Castillo,.(2025). Bioinformatics perspectives on transcriptomics: A comprehensive review of bulk and single-cell RNA sequencing analyses.

[4] Feiyang Ma , Matteo Pellegrini. (2019), ACTINN: automated identification of cell types in single cell RNA sequencing. Bioinformatics Volume 36, Issue 2, January 2020, Pages 533–538.



# Congratulations! Time to join the Community!

Congratulations on completing this tutorial notebook! If you enjoyed working through the tutorial, and want to continue working with DeepChem, we encourage you to finish the rest of the tutorials in this series. You can also help the DeepChem community in the following ways:

## Star DeepChem on [GitHub](https://github.com/deepchem/deepchem)
This helps build awareness of the DeepChem project and the tools for open source drug discovery that we're trying to build.

## Join the DeepChem Discord
The DeepChem [Discord](https://discord.gg/cGzwCdrUqS) hosts a number of scientists, developers, and enthusiasts interested in deep learning for the life sciences. Join the conversation!

In [ ]:
@manual {Bioinformatics,
 title={Identifying Cell Types in scRNA-seq Data with ACTINN and Deepchem},
 organization={DeepChem},
 author={Harindhar, Rakshit, Bharath},
 howpublished = {\url{https://github.com/deepchem/deepchem/blob/master/examples/tutorials/Cell_type_identification_using_scRNAseq_data.ipynb.ipynb}},
 year={2025},
}